Задачи. 
1. Зависит ли вероятность оплаты от выбранного пользователем уровня сложности?
2. Существует ли разница во времени между событиями регистрации и оплаты для разных групп пользователей с разным уровнем сложности?

In [2]:
import pandas as pd
events = pd.read_csv('data/7_4_Events.csv', sep = ',')
purchase = pd.read_csv('data/purchase.csv', sep = ',')

Приступим к решению первой задачи.
Отфильтруем базы данных и оставим только пользователей зарегистрированных в 2018 году. Скорректируем форматы дат:

In [3]:
#Выбем только тех пользоватей, что зарегистрировались в 2018 году:
condition = (events.start_time>='2018-01-01') & (events.start_time<'2019-01-01') & (events.event_type=='registration')
#Создадим список событий, которые произошли в 2018 году:
registered = events[condition]['user_id'].to_list()
events_2018 = events[events.user_id.isin(registered)]
#Изменим колонку start_time на формат data:
events_2018.start_time = pd.to_datetime(events_2018.start_time, format='%Y-%m-%dT%H:%M:%S')

purchase_2018 = purchase[purchase['user_id'].isin(registered)]
#Изменим колонку event_datetime на формат data:
purchase_2018.event_datetime = pd.to_datetime(purchase_2018.event_datetime, format='%Y-%m-%dT%H:%M:%S')

C:\Users\Raptor\AppData\Local\Temp\ipykernel_11396\2020182657.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_2018.start_time = pd.to_datetime(events_2018.start_time, format='%Y-%m-%dT%H:%M:%S')
C:\Users\Raptor\AppData\Local\Temp\ipykernel_11396\2020182657.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchase_2018.event_datetime = pd.to_datetime(purchase_2018.event_datetime, format='%Y-%m-%dT%H:%M:%S')


Создадим 3 группы пользователей по уровню выбранной ими сложности:

In [4]:
users_with_easy_level = events_2018[events_2018["selected_level"] == "easy"]["user_id"].unique()
print("Количество пользователей, выбравших простой уровень сложности: {}".format(len(users_with_easy_level)))
users_with_medium_level = events_2018[events_2018["selected_level"] == "medium"]["user_id"].unique()
print("Количество пользователей, выбравших средний уровень сложности: {}".format(len(users_with_medium_level)))
users_with_hard_level = events_2018[events_2018["selected_level"] == "hard"]["user_id"].unique()
print("Количество пользователей, выбравших трудный уровень сложности: {}".format(len(users_with_hard_level)))

Количество пользователей, выбравших простой уровень сложности: 2448
Количество пользователей, выбравших средний уровень сложности: 4645
Количество пользователей, выбравших трудный уровень сложности: 1249


Найдём количество пользователей с оплатой в каждой из трех групп и их средний чек:

In [30]:
purchase_df_easy = purchase_2018[purchase_2018["user_id"].isin(users_with_easy_level)]
print("Количество покупателей, выбравших простой уровень сложности: {}".format(len(purchase_df_easy)))
purchase_df_medium = purchase_2018[purchase_2018["user_id"].isin(users_with_medium_level)]
print("Количество покупателей, выбравших средний уровень сложности: {}".format(len(purchase_df_medium)))
purchase_df_hard = purchase_2018[purchase_2018["user_id"].isin(users_with_hard_level)]
print("Количество покупателей, выбравших трудный уровень сложности: {}".format(len(purchase_df_hard)))
print("Средний чек, выбравших простой уровень сложности: {}".format(purchase_df_easy['amount'].mean()))
print("Средний чек, выбравших средний уровень сложности: {}".format(purchase_df_medium['amount'].mean()))
print("Средний чек, выбравших трудный уровень сложности: {}".format(purchase_df_hard['amount'].mean()))

Количество покупателей, выбравших простой уровень сложности: 189
Количество покупателей, выбравших средний уровень сложности: 969
Количество покупателей, выбравших трудный уровень сложности: 442
Средний чек, выбравших простой уровень сложности: 114.94708994708995
Средний чек, выбравших средний уровень сложности: 109.52012383900929
Средний чек, выбравших трудный уровень сложности: 111.59502262443439


Проверим нашу первую гипотезу о связи выбора уровня сложности с количеством оплат:

In [31]:
print("Процент пользователей, которые оплатили (от числа пользователей, выбравших простой уровень сложности): {:.2%}"
      .format(len(purchase_df_easy)/len(users_with_easy_level)))
print("Процент пользователей, которые оплатили (от числа пользователей, выбравших средний уровень сложности): {:.2%}"
      .format(len(purchase_df_medium)/len(users_with_medium_level)))
print("Процент пользователей, которые оплатили (от числа пользователей, выбравших трудный уровень сложности): {:.2%}"
      .format(len(purchase_df_hard)/len(users_with_hard_level)))

Процент пользователей, которые оплатили (от числа пользователей, выбравших простой уровень сложности): 7.72%
Процент пользователей, которые оплатили (от числа пользователей, выбравших средний уровень сложности): 20.86%
Процент пользователей, которые оплатили (от числа пользователей, выбравших трудный уровень сложности): 35.39%


Вывод: имеется очевидная закономерность труднее, чем выше уровень сложности выбирает пользователь, тем чаще происходит покупка в приложении.

Приступим к решению второй аналитической задачи. Проверим зависит ли время между регистрацией и оплатой от выбора уровня сложности в игре.

Получим список пользователей, совершивших покупку, для каждой из 3 групп:

In [32]:
purchase_df_easy = purchase_2018[purchase_2018["user_id"].isin(users_with_easy_level)]
purchase_df_medium = purchase_2018[purchase_2018["user_id"].isin(users_with_medium_level)]
purchase_df_hard = purchase_2018[purchase_2018["user_id"].isin(users_with_hard_level)]
purchase_df_easy.head()

,id,user_id,event_datetime,amount
1180,16854,27884,2018-01-08 19:37:34,150
1191,16865,28182,2018-01-12 02:46:01,200
1193,16867,28207,2018-01-12 21:00:24,150
1199,16873,28090,2018-01-15 23:42:55,100
1206,16880,28378,2018-01-18 02:11:41,100


Оставим в таблицах только необходимые для вычислений столбцы - user_id и время покупки:

In [33]:
purchase_df_easy = purchase_df_easy[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "purchase_time_for_easy"}
)
purchase_df_medium = purchase_df_medium[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "purchase_time_for_medium"}
)
purchase_df_hard = purchase_df_hard[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "purchase_time_for_hard"}
)
purchase_df_easy.head()

,user_id,purchase_time_for_easy
1180,27884,2018-01-08 19:37:34
1191,28182,2018-01-12 02:46:01
1193,28207,2018-01-12 21:00:24
1199,28090,2018-01-15 23:42:55
1206,28378,2018-01-18 02:11:41


Сделаем объединенный датафрейм из событий и оплат:

In [34]:
purchase_2018['event_type'] = 'purchase'
events_df1 = events_2018.rename(columns={"id": "event_id"})
purchase_df = purchase_2018.rename(columns={"id": "purchase_id"})
total_events_df = pd.concat([events_2018,purchase_df],sort=False)
total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')
total_events_df

C:\Users\Pavel\AppData\Local\Temp\ipykernel_7508\336549344.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchase_2018['event_type'] = 'purchase'


,id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,event_datetime,amount
0,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaT,NaN
1,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaT,NaN
2,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaT,NaN
3,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaT,NaN
4,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...
68554,NaN,purchase,NaN,NaT,NaN,47498,18441.0,2019-01-02 03:48:19,100.0
68555,NaN,purchase,NaN,NaT,NaN,47647,18442.0,2019-01-02 23:26:26,150.0
68556,NaN,purchase,NaN,NaT,NaN,47554,18443.0,2019-01-03 00:36:36,50.0
68557,NaN,purchase,NaN,NaT,NaN,47742,18448.0,2019-01-04 12:51:41,50.0


Создадим вспомогательный датафрейм с датами регистраций всех пользователей:

In [35]:
registration_df = total_events_df[total_events_df['event_type'] == 'registration']
registration_df = registration_df[["user_id", "start_time"]].rename(
    columns={"start_time": "registration_time"}
)
registration_df.head()

,user_id,registration_time
0,27832,2018-01-01 03:48:40
1,27833,2018-01-01 04:07:25
2,27834,2018-01-01 08:35:10
3,27835,2018-01-01 11:54:47
4,27836,2018-01-01 13:28:07


Объединим каждый из трёх датасетов с продажами для разных уровней сложности с датасетом регистраций:

In [36]:
merged_df_easy = registration_df.merge(
    purchase_df_easy, on="user_id", how="inner"
)
merged_df_medium = registration_df.merge(
    purchase_df_medium, on="user_id", how="inner"
)
merged_df_hard = registration_df.merge(
    purchase_df_hard, on="user_id", how="inner"
)
merged_df_easy.head()

,user_id,registration_time,purchase_time_for_easy
0,27884,2018-01-04 11:50:43,2018-01-08 19:37:34
1,28090,2018-01-09 19:31:24,2018-01-15 23:42:55
2,28182,2018-01-11 10:12:20,2018-01-12 02:46:01
3,28207,2018-01-11 16:27:37,2018-01-12 21:00:24
4,28247,2018-01-12 10:01:12,2018-01-18 18:32:05


Вычислим среднее время между регистрацией и покупкой для каждой группы пользователей:

In [37]:
merged_df_easy["timedelta"] = (
    merged_df_easy["purchase_time_for_easy"] - merged_df_easy["registration_time"]
)
print("Данные для пользователей, выбравших простой уровень сложности")
merged_df_easy['timedelta'].describe()

Данные для пользователей, выбравших простой уровень сложности


count                          189
mean     3 days 22:10:23.211640211
std      2 days 07:14:41.062010764
min                0 days 04:36:58
25%                2 days 01:12:12
50%                3 days 11:00:23
75%                5 days 10:24:59
max               11 days 00:35:04
Name: timedelta, dtype: object

In [38]:
merged_df_medium["timedelta"] = (
    merged_df_medium["purchase_time_for_medium"] - merged_df_medium["registration_time"]
)
print("Данные для пользователей, выбравших средний уровень сложности")
merged_df_medium['timedelta'].describe()

Данные для пользователей, выбравших средний уровень сложности


count                          969
mean     4 days 06:12:06.576883384
std      2 days 06:25:57.480868026
min                0 days 08:39:24
25%                2 days 08:46:51
50%                4 days 03:35:26
75%                5 days 23:51:27
max               10 days 20:34:02
Name: timedelta, dtype: object

In [39]:
merged_df_hard["timedelta"] = (
    merged_df_hard["purchase_time_for_hard"] - merged_df_hard["registration_time"]
)
print("Данные для пользователей, выбравших трудный уровень сложности")
merged_df_hard['timedelta'].describe()

Данные для пользователей, выбравших трудный уровень сложности


count                          442
mean     3 days 14:55:19.257918552
std      1 days 22:22:52.441896774
min                0 days 09:41:39
25%         1 days 23:36:25.500000
50%         3 days 10:10:04.500000
75%         5 days 03:30:07.750000
max                8 days 14:21:29
Name: timedelta, dtype: object

Вывод: существенных различий между группами не наблюдается, меньше всех тратят времени на принтяие решение о покупке пользователивыбравшие сложный уровень, больше остальных тратят времени на принтяие решение о покупке пользователи выбравшие средний уровень.